## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Clyde River,CA,70.4692,-68.5914,5.11,92,86,2.17,overcast clouds
1,1,Ndele,CF,8.4109,20.6473,81.55,42,90,3.49,overcast clouds
2,2,Rorvik,NO,64.8621,11.2373,33.91,75,75,20.71,broken clouds
3,3,Okhotsk,RU,59.3833,143.3000,15.21,93,93,6.80,overcast clouds
4,4,Louis Trichardt,ZA,-23.0439,29.9032,61.30,92,94,13.91,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 80
What is the maximum temperature you would like for your trip? 100


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Ndele,CF,8.4109,20.6473,81.55,42,90,3.49,overcast clouds
10,10,Salalah,OM,17.0151,54.0924,80.69,83,0,2.30,clear sky
17,17,Oistins,BB,13.0667,-59.5333,82.49,61,20,17.27,few clouds
25,25,Puerto Ayora,EC,-0.7393,-90.3518,84.16,77,55,9.84,broken clouds
26,26,Avarua,CK,-21.2078,-159.7750,84.25,79,44,10.36,scattered clouds
31,31,Areia Branca,BR,-4.9561,-37.1369,81.03,79,55,11.23,broken clouds
32,32,Cabo San Lucas,MX,22.8909,-109.9124,86.61,60,2,10.45,clear sky
42,42,Sarangani,PH,5.4033,125.4636,82.11,79,100,23.82,light rain
52,52,Georgetown,MY,5.4112,100.3354,80.53,83,20,0.00,thunderstorm
54,54,Roebourne,AU,-20.7833,117.1333,82.65,50,38,11.50,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                123
City                   123
Country                123
Lat                    123
Lng                    123
Max Temp               123
Humidity               123
Cloudiness             123
Wind Speed             123
Current Description    123
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df                      # Create duplicate of data
clean_df.dropna(inplace = True)                      # Remove rows with NaN
clean_df.count() 

C:\Users\sarah\anaconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


City_ID                123
City                   123
Country                123
Lat                    123
Lng                    123
Max Temp               123
Humidity               123
Cloudiness             123
Wind Speed             123
Current Description    123
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Ndele,CF,81.55,overcast clouds,8.4109,20.6473,
10,Salalah,OM,80.69,clear sky,17.0151,54.0924,
17,Oistins,BB,82.49,few clouds,13.0667,-59.5333,
25,Puerto Ayora,EC,84.16,broken clouds,-0.7393,-90.3518,
26,Avarua,CK,84.25,scattered clouds,-21.2078,-159.7750,
31,Areia Branca,BR,81.03,broken clouds,-4.9561,-37.1369,
32,Cabo San Lucas,MX,86.61,clear sky,22.8909,-109.9124,
42,Sarangani,PH,82.11,light rain,5.4033,125.4636,
52,Georgetown,MY,80.53,thunderstorm,5.4112,100.3354,
54,Roebourne,AU,82.65,scattered clouds,-20.7833,117.1333,


In [8]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping")
        

Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping


In [19]:
# 7. Drop the rows where there is no Hotel Name
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_hotel_df = hotel_df                      # Create duplicate of data
clean_hotel_df.dropna(inplace = True)                      # Remove rows with NaN
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Ndele,CF,81.55,overcast clouds,8.4109,20.6473,
10,Salalah,OM,80.69,clear sky,17.0151,54.0924,"HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel"
17,Oistins,BB,82.49,few clouds,13.0667,-59.5333,Butterfly Beach Hotel
25,Puerto Ayora,EC,84.16,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
26,Avarua,CK,84.25,scattered clouds,-21.2078,-159.7750,Paradise Inn
31,Areia Branca,BR,81.03,broken clouds,-4.9561,-37.1369,Areia Branca Apart Hotel
32,Cabo San Lucas,MX,86.61,clear sky,22.8909,-109.9124,Hotel Tesoro Los Cabos
42,Sarangani,PH,82.11,light rain,5.4033,125.4636,Ballistic Islet
52,Georgetown,MY,80.53,thunderstorm,5.4112,100.3354,Cititel Penang
54,Roebourne,AU,82.65,scattered clouds,-20.7833,117.1333,Latitude20 Roebourne Village


In [20]:
# 8a. Create the output File (CSV)
output_data_file = "weatherpy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [21]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [22]:
# 11a. Add a marker layer for each city to the map. 
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)


# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))